In [5]:
import math
from enum import Enum

In [6]:
class PayoffType(str, Enum):
    Call = 'Call'
    Put = 'Put'

In [3]:
class EuropeanOption():
    def __init__(self, expiry, strike, payoffType):
        self.expiry = expiry
        self.strike = strike
        self.payoffType = payoffType
    def payoff(self, S):
        if self.payoffType == PayoffType.Call:
            return max(S - self.strike, 0)
        elif self.payoffType == PayoffType.Put:
            return max(self.strike - S, 0)
        else:
            raise Exception("payoffType not supported: ", self.payoffType)
    def valueAtNode(self, t, S, continuation):
        if continuation == None:
            return self.payoff(S)
        else:
            return continuation

In [1]:
# Trinomial Tree pricer
def trinomialPricer(S, r, q, vol, trade, n, lmda):
    t = trade.expiry / n
    u = math.exp(lmda * vol * math.sqrt(t))
    mu = r - q
    pu = 1 / 2 / lmda / lmda + (mu - vol * vol / 2) / 2 / lmda / vol * math.sqrt(t)
    pd = 1 / 2 / lmda / lmda - (mu - vol * vol / 2) / 2 / lmda / vol * math.sqrt(t)
    pm = 1 - pu - pd
    # set up the last time slice, there are 2n+1 nodes at the last time slice
    # counting from the top, the i-th node's stock price is S * u^(n - i), i from 0 to n+1
    vs = [trade.payoff(S * u ** (n - i)) for i in range(2*n + 1)]
    # iterate backward
    for i in range(n - 1, -1, -1):
        # calculate the value of each node at time slide i, there are i nodes
        for j in range(2*i + 1):
            nodeS = S * u ** (i - j)
            continuation = math.exp(-r * t) * (vs[j] * pu +  + vs[j+1] * pm + vs[j+2] * pd)
            vs[j] = trade.valueAtNode(t * i, nodeS, continuation)
    return vs[0]


In [27]:
S, r, vol = 100, 0.01, 0.2
opt = EuropeanOption(1, 105, PayoffType.Call)
prc = trinomialPricer(S, r, 0, vol, opt, 1000, math.sqrt(3))
print("Trinomial tree Price = \t ", prc)

Trinomial tree Price = 	  6.2982090564185835


In [24]:
n = 2
t = opt.expiry/n
lmda = 2.623144635296316
u = math.exp(lmda * vol * math.sqrt(t))
vs = [opt.payoff(S * u ** (n - i)) for i in range(2*n + 1)]
vs

[104.99999981752637, 39.91376739893499, 0, 0, 0]

In [22]:
def calibrate_lambda(S, K, vol, T, n, initial_lambda, tolerance, trade):
    lambda_min, lambda_max = initial_lambda, initial_lambda * 2  # Set a range for lambda
    lambda_calibrated = initial_lambda
    node_difference = float('inf')
    t = T/n

    while abs(node_difference) > tolerance:
        # Calculate the up and down factors
        u = math.exp(lambda_calibrated * vol * math.sqrt(t))

        # Build the final layer of the tree
        vs = [trade.payoff(S * u ** (n - i)) for i in range(2*n+1)]

        # Find the node closest to the strike price K
        closest_node = min(vs, key=lambda x: abs(x-K))
        node_difference = closest_node - K

        # Adjust lambda
        if node_difference > 0:
            lambda_max = lambda_calibrated
        else:
            lambda_min = lambda_calibrated

        lambda_calibrated = (lambda_max + lambda_min) / 2  # New lambda is the midpoint

    return lambda_calibrated

In [23]:
calibrate_lambda(100, 105, 0.2, 1, 2, math.sqrt(3), 1e-8, opt)

2.623144635296316